## One-unit neural network model for logistic regression

In this notebook, we illustrate how to do [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression)
using a neural network-like implementation with one single unit (perceptron).
This is equivalent to a logistic regression model, only it is solved using neural networks (forward and backpropagation) rather than with maximum likelihood (or similar algorithms).

As example data, we use the famous [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)

Basically, a neural network implementation of logistic regression will look like the sketch below: first, the vector of input features $\mathbf{x}$ is multiplied by the vector of weights $\mathbf{w}$, the results are summed up together and the bias term $b$ is added. This will return the real-valued variable $z$ in the interval $[\pm ∞]$.
Then, $z$ is activated with the logistic (sigmoid) function $\rightarrow \sigma(z)$ to give $P(y=1|x)$, the probability of belonging to class `1` given the input features $x$.

<img src="logistic_perceptron.png" alt="perceptron" style="width: 500px;"/>

## Loading libraries and setting the random seed

First of all, we load some necessary libraries; then we setup the random seed to ensure reproducibility of results. Since tensorflow uses an internal random generator we need to fix both the general seed (via numpy `seed()`) and tensorflow seed (via `set_seet()`)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

  # Set the seed using keras.utils.set_random_seed. This will set:
  # 1) `numpy` seed
  # 2) `tensorflow` random seed
  # 3) `python` random seed
tf.keras.utils.set_random_seed(10)

  # This will make TensorFlow ops as deterministic as possible, but it will
  # affect the overall performance, so it's not enabled by default.
  # `enable_op_determinism()` is introduced in TensorFlow 2.9.
tf.config.experimental.enable_op_determinism()

## Get the data

From `sklearn.datasets` there are generally two ways to import the data:

1. `return_X_y = False` (default): returns a "bunch" object that contains both the `target` and the `features` (to be accessed as attributes): `<dataset>.target` and `<dataset>.data`
2. `return_X_y = True`: returns directly the target and features separately

In [ ]:
import sklearn.datasets

## 1)
iris = sklearn.datasets.load_iris()
features = iris.data
target = iris.target

In [ ]:
## 2)
(features_, target_) = sklearn.datasets.load_iris(return_X_y = True) ## feature names are not returned

In [ ]:
target == target_

Next, we convert data and features (originally as `numpy` arrays) to `pandas` dataframes / series

In [ ]:
iris.data = pd.DataFrame(features, columns=iris.feature_names) #converting numpy array -> pandas DataFrame
iris.target = pd.Series(target) #converting numpy array -> pandas Series

The **iris** dataset is a historic dataset first used by **Ronald Fisher** in his 1936 paper on linear discriminant analysis (LDA).
The dataset contains information on 150 flower samples, belonging to three species of Iris (Iris setosa, Iris virginica and Iris versicolor: the `target`).
Four `features` were measured from each sample: the length and the width of the sepals and of the petals, in centimeters.

In [ ]:
print(iris.DESCR)

## Exploratory Data Analysis (EDA)

As said, the feature data has 150 rown and 4 columns:

In [ ]:
print('Shape of the feature table: ' + str(iris.data.shape))

The four features are real-valued numbers in the $10^1$ range:

In [ ]:
iris.data.describe()

The `target` is a vector (`pandas` series) of integers (0, 1, 2), one for each flower species (50 samples each):

In [ ]:
#using Counter object to print a tally of the classes
from collections import Counter
print('Numerosity for each class: ' + str(Counter(iris.target)))

Classes are represented via a numeric index: 0 for *setosa*, 1 for *versicolor*, 2 for *virginica*. The samples are in order: the first 50 samples are *setosa*, then 50 *versicolor* and the last 50 are *virginica*.

When working with a new dataset, it is always importat to plot the data. We are unfortunately talking about a 5-dimensional dataset (the four features + the target class) which is not easily representable.
One possibility is to take a slice (a subset) of the whole dataset.

In the next code chunk we plot two features plus the class.

In [ ]:
## SELECT FEATURES TO PLOT
#change these two values to plot different features, remembering the numbering:
# 0 : sepal length (cm)
# 1 : sepal width (cm)
# 2 : petal length (cm)
# 3 : petal width (cm)
feature_x = 0
feature_y = 1

#starting a new plot
fig, ax = plt.subplots()

#adding data in three bunches of 50, once per class
ax.scatter(x=iris.data.iloc[0:50,feature_x],    y=iris.data.iloc[0:50,feature_y],    c='red',   label=iris.target_names[0])
ax.scatter(x=iris.data.iloc[50:100,feature_x],  y=iris.data.iloc[50:100,feature_y],  c='green', label=iris.target_names[1])
ax.scatter(x=iris.data.iloc[100:150,feature_x], y=iris.data.iloc[100:150,feature_y], c='blue',  label=iris.target_names[2])

#the axis names are taken from feature names
ax.set_xlabel(iris.feature_names[feature_x])
ax.set_ylabel(iris.feature_names[feature_y])

#adding the legend and printing the plot
ax.legend()
plt.show()

The plot shows clearly that setosa is quite separate from the other two classes. Whichever features you choose for the plot, you more or less get this same general result.

This is however a **three-class** problem: however, we want to build a **neural network model** for **logistic regression** applied to a **binary classification problem**.

Therefore, we need some preparatory ('preprocessing') steps.

## Data preprocessing

We need to transform a **three-class problem in a two-class problem**: we choose to **join together Setosa and Versicolor** (the red and green dots from the plot above), which appear to be less obviously separable based on sepal (and petal) length and width (and hence promise to provide more of a challenge to our neural network problem).
In other words, we want to build a classifier able to discriminate `virginica` (which becomes the new class "1") from the other `non-virginica` flowers (which all together become the new class "0").

To make the problem a little more challenging (and interesting) we decide to **drop half of the features**, **using only the first two** columns.

In [ ]:
#simplifly the problem: less classes, less features
features = iris.data.iloc[:, 0:2]
target = iris.target

#updating class labels. To makes things difficult we put together old classes 0 and 1
#in a new class (non virginica) and keep old class 2 (virginica) as new class 1.
#For an easier problems put together versicolor and virginica and keep setosa by itself
j = 100 ## split: 50 for setosa vs versicolor+virginica, 100 for setosa+versicolor vs virginica
target[0:j] = 0
target[j:150] = 1

In [ ]:
#starting a new plot
fig, ax = plt.subplots()

#adding data in two bunches
ax.scatter(x=features.iloc[0:j,0],   y=features.iloc[0:j,1],   c='red',  label='non-virginica')
ax.scatter(x=features.iloc[j:150,0], y=features.iloc[j:150,1], c='blue', label='virginica')

#the axis names are taken from feature names
ax.set_xlabel(iris.feature_names[feature_x])
ax.set_ylabel(iris.feature_names[feature_y])

# displaying the title
plt.title("Two-class simplified problem")

#adding the legend and printing the plot
ax.legend()
plt.show()

Things are getting interesting! This is now a difficult problem and there is no clear cut solution. Let's proceed.

## Training and validation sets

Each time there is some kind of "learning" involved we need to split our data. A subset will be used for training, and a subset will be used for validation.

In our current dataset the samples are sorted by class: the first 100 are "non-virginica" and the remaining 50 are "virginica".
We want to keep this 2:1 proportion (roughly) the same in both train and validation set.

Obviously (given the ordered data), taking the first $80\%$ (120 samples) of the data as training and the rest (30 samples) as validation would be a bad choice ... (only 'virginica' samples in the validation set, 4:1 class proportion in the training set).   

Simple random sampling would also not be a good solution, since the 2:1 proportion will not be maintained.

Therefore we are going to use what is called a [stratified approach](https://machinelearningmastery.com/cross-validation-for-imbalanced-classification/) using a [StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html) object from `scikit-learn`:


In [ ]:
#we want to have the same proportion of classes in both train and validation sets
from sklearn.model_selection import StratifiedShuffleSplit

#building a StratifiedShuffleSplit object (sss among friends) with 20% data
#assigned to validation set (here called "test")
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

#the .split() method returns (an iterable over) two lists which can be
#used to index the samples that go into train and validation sets
for train_index, val_index in sss.split(features, target):
    features_train = features.iloc[train_index, :]
    features_val   = features.iloc[val_index, :]
    target_train   = target[train_index]
    target_val     = target[val_index]

#let's print some shapes to get an idea of the resulting data structure
print(features_train.shape)
print(features_val.shape)
print(target_train.shape)
print(target_val.shape)

In [ ]:
print(Counter(target_train))
print(Counter(target_val))

We see that we have $80\%$ data ready for training and $20\%$ for validation, both sets with a 2:1 target class ratio: job done!

## Build the neural network model

We want to use a neural network model for logistic regression.
There are many `Python` frameworks that implement (Deep) Neural Networks: here we are using `Keras` (for more details on how to use *keras* a good starting point is the [documentation on training and evaluation](https://www.tensorflow.org/guide/keras/train_and_evaluate)).

Our neural network will be very minimal, as illustrated in the sketch at the beginning of this notebook: it will be comprised of only one node (neuron) that will perform both 1) the linear combination of weighted input variables + bias term; and 2) activate the result of step 1 with the sigmoid function to produce the probability of belonging to class "1" (or "0").

We are now ready to build the NN (neural network) model!

- model set-up
- model architecture and compiling

### Model set-up

We first need to define some hyperparameters:

1. activation function (`sigmoid`, this is a binary classification problem)
2. optimizer (the algorithm that carries out forward and backpropagation to solve the model)
3. the loss function (`binary cross-entropy`, again this is binary classification!)

In [ ]:
activation_function = 'sigmoid'
optimizing_method = 'rmsprop'
loss_function = 'binary_crossentropy'

### Model architecture

This is a one-layer one-unit neural network model.
The neural network is fully connected, meaning that the unit receives in input all features.

In [ ]:
#we are building a "sequential" model, meaning that the data will
#flow like INPUT -> ELABORATION -> OUTPUT. In particular, we will
#not have any loops, i.e. our output will never be recycled as
#input for the first layer
from keras.models import Sequential

#a "dense" layer is a layer were all the data coming in are connected
#to all nodes (fully connected).
#In our case there is only one node in the layer, and
#it receives all the input features
from keras.layers import Dense

# 2-class logistic regression in Keras
model = Sequential()
model.add(Dense(units=1, activation='sigmoid', input_dim=features_train.shape[1]))

#the model is declared, but we still need to compile it to actually
#build all the data structures
model.compile(optimizer=optimizing_method, loss=loss_function)

With the `.summary()` method we can take a look inside the model:

In [ ]:
print(model.summary())

We see that there are three trainable parameters (the weights for the two input variables, W1, W2, plus the bias term 'B'), and a single node.
The output is a single number. Excellent.

## Training the neural network model

In [ ]:
history = model.fit(features_train, target_train, epochs=10, validation_data=(features_val, target_val))

In [ ]:
#function to take a look at losses evolution
def plot_loss_history(h, title):
    plt.plot(h.history['loss'], label = "Train loss")
    plt.plot(h.history['val_loss'], label = "Validation loss")
    plt.xlabel('Epochs')
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
plot_loss_history(history, 'Logistic (10 epochs)')